In [30]:
import psycopg2
import pytz
import datetime
import numpy as np
from matplotlib.patches import Rectangle
from pyiem.network import Table as NetworkTable
from pyiem.util import get_dbconn
from pyiem.plot import figure
nt = NetworkTable(["IA_ASOS", "NY_ASOS"])
POSTGIS = get_dbconn('postgis')
pcursor = POSTGIS.cursor()
ASOS = get_dbconn('asos')
acursor = ASOS.cursor()



In [22]:
def get_asos(sid, sts, ets):
    acursor.execute(f"""SELECT valid, vsby, sknt, coalesce(gust, sknt) as gust,
    case when gust > sknt then gust else sknt end from t{sts.year}
 WHERE station = %s and valid BETWEEN %s and %s and sknt is not null and vsby is not null
 and report_type in (3, 4) ORDER by valid ASC""",
                    (sid, sts.strftime("%Y-%m-%d %H:%M"), ets.strftime("%Y-%m-%d %H:%M")))
    valid = []
    vsby = []
    sknt = []
    gust = []
    for row in acursor:
        valid.append( row[0]) 
        vsby.append( row[1] )
        sknt.append( row[2] )
        gust.append( row[3] * 1.15)
        
    return valid, vsby, sknt, gust

In [38]:
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle

central = pytz.timezone("America/Chicago")

fig = figure(
    title="Airport Observations During 22-24 Dec 2022 Blizzard Warning",
    subtitle="Five selected Iowa Airports and Buffalo, NY for a comparison.",
    figsize=(12.80, 10.24))
ax = [
    fig.add_axes([0.07, 0.65, 0.38, 0.2]),
    fig.add_axes([0.07, 0.37, 0.38, 0.2]),
    fig.add_axes([0.07, 0.1, 0.38, 0.2]),
    fig.add_axes([0.56, 0.65, 0.38, 0.2]),
    fig.add_axes([0.56, 0.37, 0.38, 0.2]),
    fig.add_axes([0.56, 0.1, 0.38, 0.2]),
]

for i, sid in enumerate(['MCW', 'IFA', 'AMW', 'EST', 'CID', 'BUF']):
    pcursor.execute("""select issue, expire from warnings where ugc = %s
      and phenomena = 'BZ' and significance = 'W' 
      and expire > issue and issue > '2022-12-10' ORDER by issue DESC""",
                       (nt.sts[sid]["ugc_zone"], )
    )
    row = pcursor.fetchone()
    valid, vsby, sknt, gust = get_asos(sid, row[0], row[1])
    print(i, len(valid))
    ny = "** New York **" if sid == 'BUF' else ''
    ax[i].text(0, 1.04, f"[{sid}] {nt.sts[sid]['name']} {ny}", transform=ax[i].transAxes, fontsize=14)
    ax[i].scatter(valid, vsby, marker='o', s=40, color='b', zorder=2)
    ax[i].set_ylabel("Visibility [mile]", color='b')
    ax2 = ax[i].twinx()
    ax2.scatter(valid, gust, marker='o', s=40, color='r', zorder=2)
    ax2.set_ylabel("Wind Speed/Gust [mph]", color='r')
    ax[i].set_xlim(min(valid), max(valid))
    ax[i].set_ylim(0, 10.1)
    ax2.set_ylim(0, 80)
    
    ax[i].xaxis.set_major_locator(
                               mdates.DayLocator(interval=1,
                                                 tz=central)
                               )
    ax[i].xaxis.set_major_formatter(mdates.DateFormatter('|\n%d %b %Y',
                                                      tz=central))

    ax[i].xaxis.set_minor_locator(
                               mdates.HourLocator(interval=5,
                                                 tz=central)
                               )
    ax[i].xaxis.set_minor_formatter(mdates.DateFormatter('%I %p',
                                                      tz=central))
    ax[i].grid(True)
    ax[i].set_yticks(np.linspace(0, 10, 5))
    ax2.set_yticks(np.linspace(0, 80, 5))
    ax2.axhline(35, linestyle='-.', color='r')
    ax[i].axhline(0.25, linestyle='-.', color='b')

    hit = None
    for j in range(1,len(valid)):
        if vsby[j] <= 0.25 and gust[j] >= 35:
            if hit is None:
                hit = j - 1
                continue
            if hit is not None:
                continue
        else:
            if hit is None:
                continue
            secs = (valid[j]-valid[hit]).total_seconds()
            color = '#EEEEEE' if secs < (3*3600.) else 'lightblue'
            if color == 'lightblue':
                print(secs, color, valid[0])
            rect = Rectangle((valid[hit],0), datetime.timedelta(seconds=secs) , 60, fc=color, zorder=1, ec='None') 
            ax[i].add_patch(rect)
            hit = None
    if hit:
        secs = (valid[j]-valid[hit]).total_seconds()
        color = '#EEEEEE' if secs < (3*3600.) else 'lightblue'
        if color == 'lightblue':
            print(secs, color, valid[0])
        rect = Rectangle((valid[hit],0), datetime.timedelta(seconds=secs) , 60, fc=color, zorder=1, ec='None') 
        ax[i].add_patch(rect)
        hit = None

fig.text(0.5, 0.04, "Gray Shaded areas <=1/4 mile vis & 35+ MPH winds, Light Blue >= 3 Hours", ha="center")
fig.savefig('221224.png')
plt.close()

0 117
11280.0 lightblue 2022-12-22 12:19:00-06:00
1 125
21600.0 lightblue 2022-12-22 12:15:00-06:00
2 90
3 112
30900.0 lightblue 2022-12-22 12:15:00-06:00
4 59
5 63
85020.0 lightblue 2022-12-23 06:43:00-06:00
